## Miniproject 1 - Neural Networks Approach

In [1]:
import numpy as np 
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.core import Dense, Activation, Flatten, Dropout
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [2]:
cd E:\Caltech_Winter\CS 155\mini project1\data

E:\Caltech_Winter\CS 155\mini project1\data


## Import Training Data

In [6]:
from numpy import genfromtxt
import numpy as np
train_file = 'training_data.txt'
test_file = 'test_data.txt'
train_data = genfromtxt(train_file, delimiter=' ')
test_data = genfromtxt(test_file, delimiter=' ')
train_data = train_data[1:]
X_test = test_data[1:]

np.random.seed(123456)
ind = np.random.permutation(train_data.shape[0])
new_train_data = train_data[ind].astype(int)
X_train = new_train_data[:, 1:]
y_train = new_train_data[:, 0]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(20000, 1000)
(20000,)
(10000, 1000)


In [79]:
bagging_train_file = './bagging_pca.txt'
bagging_train_data = genfromtxt(bagging_train_file, delimiter='\t')
bagging_train_data = bagging_train_data[1:]
bagging_X_train = bagging_train_data[:, 1:]
bagging_y_train = bagging_train_data[:, 0]

train_file = './training_data.txt'
train_data = genfromtxt(train_file, delimiter=' ')
train_data = train_data[1:]
np.random.seed(123456)
ind = np.random.permutation(train_data.shape[0])
new_train_data = train_data[ind].astype(int)
X_train = new_train_data[:, 1:]
y_train = new_train_data[:, 0]

X_train = np.concatenate([bagging_X_train, X_train], axis = 1)
print(np.sum(y_train != bagging_y_train))
print(X_train.shape)

0
(20000, 1015)


In [116]:
X_train = train_data[:, 1:]
y_train = train_data[:, 0]
print(X_train.shape)

(20000, 6)


## Build Neural Networks Function

In [56]:
def NeuralRun(X_train, y_train, num_epoch, dropout, num_hidden_x, learning_rate):
    model = Sequential()
    model.add(Dense(num_hidden_x[0], input_shape=(len(X_train[0]),)))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout[0]))
    
    model.add(Dense(num_hidden_x[1]))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout[1]))
    
    model.add(Dense(num_hidden_x[2]))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout[2]))
    
    model.add(Dense(num_hidden_x[3]))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout[3]))
    
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    # compile network
    Adam = keras.optimizers.Adam(lr = learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=Adam, metrics=['accuracy'])

    fit = model.fit(X_train, y_train, batch_size = 32, epochs=num_epoch, verbose=1)

    return model

## Cross Validation Function

In [31]:
def CrossValidation(X_train, y_train, num_epoch, fold, dropout, num_hidden_x, learning_rate):

    kf = KFold(n_splits=fold)
    max_acc = 0
    i = 1
    sum_acc = 0
    
    print('Parameters: [Epochs] %d, [Fold] %d; [Dropout] %s, [Hidden X] %s, [Learning Rate] %s' 
          % (num_epoch, fold, dropout, num_hidden_x, learning_rate))
    print('Start Training...')
    for train_index, test_index in kf.split(X_train):
        print('Train on Fold %1d' % i)
        i = i + 1
        
        model = NeuralRun(X_train[train_index], y_train[train_index], num_epoch, dropout, num_hidden_x, learning_rate, \
                          X_train[test_index], y_train[test_index])
        
        # Printing the accuracy of our model, according to the loss function specified in model.compile
        score = model.evaluate(X_train[train_index], y_train[train_index], verbose=0)
        print('Training accuracy:', score[1])
        score = model.evaluate(X_train[test_index], y_train[test_index], verbose=0)
        print('Test accuracy:', score[1])
        
        sum_acc += score[1]
        if score[1] > max_acc:
            max_model = model
            max_acc = score[1]
    
    print('Best accuracy:', max_acc, 'Avg accuracy:', sum_acc / fold)
    return max_model

## Testing for Parameters

In [55]:
fold = 5
dropout = [0.5, 0.5, 0.5, 0.5]
num_hidden_x = [200, 200, 200, 200]
learning_rate = 0.001
num_epoch = 3
model = CrossValidation(X_train, y_train, num_epoch, fold, dropout, num_hidden_x, learning_rate)

Parameters: [Epochs] 3, [Fold] 5; [Dropout] [0.5, 0.5, 0.5, 0.5], [Hidden X] [200, 200, 200, 200], [Learning Rate] 0.001
Start Training...
Train on Fold 1
Train on 16000 samples, validate on 4000 samples
Epoch 1/3
16000/16000 [==============================] - 23s 1ms/step - loss: 0.5191 - acc: 0.7306 - val_loss: 0.3631 - val_acc: 0.8482
Epoch 2/3
16000/16000 [==============================] - 17s 1ms/step - loss: 0.3553 - acc: 0.8531 - val_loss: 0.3510 - val_acc: 0.8522
Epoch 3/3
16000/16000 [==============================] - 16s 1ms/step - loss: 0.2999 - acc: 0.8818 - val_loss: 0.3455 - val_acc: 0.8542
Training accuracy: 0.92675
Test accuracy: 0.85425
Train on Fold 2
Train on 16000 samples, validate on 4000 samples
Epoch 1/3
16000/16000 [==============================] - 26s 2ms/step - loss: 0.5039 - acc: 0.7468 - val_loss: 0.3667 - val_acc: 0.8482
Epoch 2/3
16000/16000 [==============================] - 19s 1ms/step - loss: 0.3495 - acc: 0.8560 - val_loss: 0.3622 - val_acc: 0.8485
E

## Try Bagging

In [46]:
train_file = 'training_data.txt'
test_file = 'test_data.txt'
train_data = genfromtxt(train_file, delimiter=' ')
test_data = genfromtxt(test_file, delimiter=' ')
train_data = train_data[1:]

np.random.seed(123456)
ind = np.random.permutation(train_data.shape[0])
new_train_data = train_data[ind].astype(int)

# Here, y = 0 means bad review, and y = 1 means good review
X_train = new_train_data[:, 1:]
y_train = new_train_data[:, 0]
y_stack = np.zeros((len(y_train), 1))

In [47]:
fold = 5
dropout = 0.8
num_hidden_x = 80
learning_rate = 0.001
num_epoch = 10
kf = KFold(n_splits=fold)

for train_index, test_index in kf.split(X_train):
    model = NeuralRun(X_train[train_index], y_train[train_index], num_epoch, dropout, num_hidden_x, learning_rate)
    y_stack[test_index] = model.predict(X_train[test_index])

In [51]:
y_stack_int = [1 if i > 0.5 else 0 for i in y_stack]

with open("NN.txt", 'a') as text_file:
    text_file.write("Id,Prediction\n")
    for i in range(len(y_stack_int)):
        text_file.write("{},{}\n".format(i+1, y_stack_int[i]))

20000


## Prediction in test

In [57]:
fold = 5
dropout = [0.65, 0.55, 0.45, 0.35]
num_hidden_x = [200, 200, 200, 200]
learning_rate = 0.001
num_epoch = 3
model = NeuralRun(X_train, y_train, num_epoch, dropout, num_hidden_x, learning_rate)
y_test = model.predict(test_data[1:])

Epoch 1/3
20000/20000 [==============================] - 28s 1ms/step - loss: 0.5132 - acc: 0.7376
Epoch 2/3
20000/20000 [==============================] - 21s 1ms/step - loss: 0.3675 - acc: 0.8467
Epoch 3/3
20000/20000 [==============================] - 21s 1ms/step - loss: 0.3238 - acc: 0.8688


In [61]:
y_test = model.predict(test_data[1:])
y_test_int = [1 if i > 0.5 else 0 for i in y_test]

with open("FinalTest.txt", 'a') as text_file:
    text_file.write("Id,Prediction\n")
    for i in range(len(y_test_int)):
        text_file.write("{},{}\n".format(i+1, y_test_int[i]))

## Help for Adaboost

In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

y_stack_ada = np.zeros(len(y_train))

fold = 5
kf = KFold(n_splits=fold)
i = 0

for train_index, test_index in kf.split(X_train):
    if i > 2:
        break
    i = i + 1
    print('New Fold Starts...')
    model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4), algorithm="SAMME", n_estimators=1500, learning_rate=1.0)
    model.fit(X_train[train_index], y_train[train_index])
    y_stack_ada[test_index] = model.predict(X_train[test_index])
    print('1 Fold Complete...')

print(y_stack_ada)

New Fold Starts...
1 Fold Complete...
New Fold Starts...
1 Fold Complete...
New Fold Starts...
1 Fold Complete...
[0. 0. 1. ... 0. 0. 0.]


In [19]:
y_stack_int = [1 if i > 0.5 else 0 for i in y_stack_ada]

with open("AdaBoost.txt", 'a') as text_file:
    text_file.write("Id,Prediction\n")
    for i in range(len(y_stack_int)):
        text_file.write("{},{}\n".format(i+1, y_stack_int[i]))